In [0]:
from pyspark.sql.types import DateType

## Carregamento dos arquivos json

In [0]:
df_resumo = spark.read.json("dbfs:/FileStore/_covid_data_lake/_raw/_summary/summary_paises.json")
df_pais = spark.read.json("dbfs:/FileStore/_covid_data_lake/_raw/_pais/pais.json")
df_dados_paises = spark.read.json("dbfs:/FileStore/_covid_data_lake/_raw/_dados_paises/dados_paises.json")

### Resumo estatístico das tabelas dados_paises e resumo (summary)

In [0]:
display(df_resumo.select('NEWCONFIRMED', 'NEWDEATHS', 'NEWRECOVERED', 'TOTALCONFIRMED', 'TOTALDEATHS', 'TOTALRECOVERED')\
        .describe())

summary,NEWCONFIRMED,NEWDEATHS,NEWRECOVERED,TOTALCONFIRMED,TOTALDEATHS,TOTALRECOVERED
count,190,190,190,190,190,190
mean,2461.0105263157893,50.94736842105263,1238.4789473684211,620814.5842105263,13777.857894736842,351794.5789473684
stddev,8162.603986544886,214.61055504553707,3997.9161406673356,2481582.6010728846,49210.16894874296,1173407.4292026402
min,0,0,0,1,0,0
max,79876,2286,43428,29236735,529263,10938146


In [0]:
display(df_dados_paises.select('ACTIVE', 'CONFIRMED', 'DEATHS', 'RECOVERED')\
        .describe())

summary,ACTIVE,CONFIRMED,DEATHS,RECOVERED
count,78660,78660,78660,78660
mean,72665.01670480549,187597.5742817188,4821.271217899822,110917.69894482584
stddev,862209.7110509691,1121032.0270692352,23423.73552445359,596277.597001509
min,0,0,0,0
max,28624522,29236735,529263,10938146


In [0]:
display(df_dados_paises)
display(df_pais)
display(df_resumo)

ACTIVE,CONFIRMED,DATE,DEATHS,ID,ID_PAIS,LAT,LON,RECOVERED
0,0,2020-01-22T00:00:00.000Z,0,78661,670,0,0,0
0,0,2020-01-23T00:00:00.000Z,0,78662,670,0,0,0
0,0,2020-01-24T00:00:00.000Z,0,78663,670,0,0,0
0,0,2020-01-25T00:00:00.000Z,0,78664,670,0,0,0
0,0,2020-01-26T00:00:00.000Z,0,78665,670,0,0,0
0,0,2020-01-27T00:00:00.000Z,0,78666,670,0,0,0
0,0,2020-01-28T00:00:00.000Z,0,78667,670,0,0,0
0,0,2020-01-29T00:00:00.000Z,0,78668,670,0,0,0
0,0,2020-01-30T00:00:00.000Z,0,78669,670,0,0,0
0,0,2020-01-31T00:00:00.000Z,0,78670,670,0,0,0


ID,NOME,SIGLA,SLUG
497,Timor-Leste,TL,timor-leste
498,Bouvet Island,BV,bouvet-island
499,Kenya,KE,kenya
500,Lithuania,LT,lithuania
501,Mali,ML,mali
502,Panama,PA,panama
503,Pitcairn,PN,pitcairn
504,Saint Kitts and Nevis,KN,saint-kitts-and-nevis
505,Lesotho,LS,lesotho
506,Nigeria,NG,nigeria


DATE,ID,ID_PAIS,NEWCONFIRMED,NEWDEATHS,NEWRECOVERED,TOTALCONFIRMED,TOTALDEATHS,TOTALRECOVERED
2021-03-12T00:45:38.873Z,381,738,18,0,90,55894,2451,49499
2021-03-12T00:45:38.873Z,382,671,631,17,714,114840,1986,77498
2021-03-12T00:45:38.873Z,383,672,138,4,109,114681,3026,79428
2021-03-12T00:45:38.873Z,384,721,41,0,23,11130,112,10708
2021-03-12T00:45:38.873Z,385,557,47,0,84,21161,516,19761
2021-03-12T00:45:38.873Z,386,618,20,1,34,882,23,489
2021-03-12T00:45:38.873Z,387,636,7693,107,5049,2169694,53359,1961640
2021-03-12T00:45:38.873Z,388,736,340,5,163,175538,3237,165259
2021-03-12T00:45:38.873Z,389,605,16,0,9,29090,909,22945
2021-03-12T00:45:38.873Z,390,655,2528,19,2012,481919,8776,449053


In [0]:
df_pais_dados_pais = df_dados_paises.join(df_pais,on=df_dados_paises.ID_PAIS == df_pais.ID)
display(df_pais_dados_pais)

ACTIVE,CONFIRMED,DATE,DEATHS,ID,ID_PAIS,LAT,LON,RECOVERED,ID,NOME,SIGLA,SLUG
0,0,2020-01-22T00:00:00.000Z,0,78661,670,0,0,0,670,Myanmar,MM,myanmar
0,0,2020-01-23T00:00:00.000Z,0,78662,670,0,0,0,670,Myanmar,MM,myanmar
0,0,2020-01-24T00:00:00.000Z,0,78663,670,0,0,0,670,Myanmar,MM,myanmar
0,0,2020-01-25T00:00:00.000Z,0,78664,670,0,0,0,670,Myanmar,MM,myanmar
0,0,2020-01-26T00:00:00.000Z,0,78665,670,0,0,0,670,Myanmar,MM,myanmar
0,0,2020-01-27T00:00:00.000Z,0,78666,670,0,0,0,670,Myanmar,MM,myanmar
0,0,2020-01-28T00:00:00.000Z,0,78667,670,0,0,0,670,Myanmar,MM,myanmar
0,0,2020-01-29T00:00:00.000Z,0,78668,670,0,0,0,670,Myanmar,MM,myanmar
0,0,2020-01-30T00:00:00.000Z,0,78669,670,0,0,0,670,Myanmar,MM,myanmar
0,0,2020-01-31T00:00:00.000Z,0,78670,670,0,0,0,670,Myanmar,MM,myanmar


In [0]:
df_pais_dados_pais = df_pais_dados_pais.drop('ID', 'ID_PAIS', 'LAT', 'LON', 'ID', 'SLUG')
df_pais_dados_pais.show(7)

+------+---------+--------------------+------+---------+-------+-----+
ACTIVE|CONFIRMED| DATE|DEATHS|RECOVERED| NOME|SIGLA|
+------+---------+--------------------+------+---------+-------+-----+
 0| 0|2020-01-22T00:00:...| 0| 0|Myanmar| MM|
 0| 0|2020-01-23T00:00:...| 0| 0|Myanmar| MM|
 0| 0|2020-01-24T00:00:...| 0| 0|Myanmar| MM|
 0| 0|2020-01-25T00:00:...| 0| 0|Myanmar| MM|
 0| 0|2020-01-26T00:00:...| 0| 0|Myanmar| MM|
 0| 0|2020-01-27T00:00:...| 0| 0|Myanmar| MM|
 0| 0|2020-01-28T00:00:...| 0| 0|Myanmar| MM|
+------+---------+--------------------+------+---------+-------+-----+
only showing top 7 rows

In [0]:
df_pais_dados_pais.printSchema()

root
-- ACTIVE: long (nullable = true)
-- CONFIRMED: long (nullable = true)
-- DATE: date (nullable = true)
-- DEATHS: long (nullable = true)
-- RECOVERED: long (nullable = true)
-- NOME: string (nullable = true)
-- SIGLA: string (nullable = true)

In [0]:
df_pais_dados_pais = df_pais_dados_pais.withColumn('DATE', df_pais_dados_pais['DATE'].cast(DateType()))

In [0]:
df_resumo.printSchema()

root
-- DATE: string (nullable = true)
-- ID: long (nullable = true)
-- ID_PAIS: long (nullable = true)
-- NEWCONFIRMED: long (nullable = true)
-- NEWDEATHS: long (nullable = true)
-- NEWRECOVERED: long (nullable = true)
-- TOTALCONFIRMED: long (nullable = true)
-- TOTALDEATHS: long (nullable = true)
-- TOTALRECOVERED: long (nullable = true)

In [0]:
df_resumo = df_resumo.withColumn('DATE', df_resumo['DATE'].cast(DateType()))

In [0]:
df_resumo_pais = df_resumo.join(df_pais,on=df_resumo.ID_PAIS == df_pais.ID)
display(df_resumo_pais)

DATE,ID,ID_PAIS,NEWCONFIRMED,NEWDEATHS,NEWRECOVERED,TOTALCONFIRMED,TOTALDEATHS,TOTALRECOVERED,ID,NOME,SIGLA,SLUG
2021-03-12,381,738,18,0,90,55894,2451,49499,738,Afghanistan,AF,afghanistan
2021-03-12,382,671,631,17,714,114840,1986,77498,671,Albania,AL,albania
2021-03-12,383,672,138,4,109,114681,3026,79428,672,Algeria,DZ,algeria
2021-03-12,384,721,41,0,23,11130,112,10708,721,Andorra,AD,andorra
2021-03-12,385,557,47,0,84,21161,516,19761,557,Angola,AO,angola
2021-03-12,386,618,20,1,34,882,23,489,618,Antigua and Barbuda,AG,antigua-and-barbuda
2021-03-12,387,636,7693,107,5049,2169694,53359,1961640,636,Argentina,AR,argentina
2021-03-12,388,736,340,5,163,175538,3237,165259,736,Armenia,AM,armenia
2021-03-12,389,605,16,0,9,29090,909,22945,605,Australia,AU,australia
2021-03-12,390,655,2528,19,2012,481919,8776,449053,655,Austria,AT,austria


In [0]:
df_resumo_pais = df_resumo_pais.drop('ID', 'ID_PAIS', 'ID', 'SLUG')

In [0]:
display(df_resumo_pais)

DATE,NEWCONFIRMED,NEWDEATHS,NEWRECOVERED,TOTALCONFIRMED,TOTALDEATHS,TOTALRECOVERED,NOME,SIGLA
2021-03-12,18,0,90,55894,2451,49499,Afghanistan,AF
2021-03-12,631,17,714,114840,1986,77498,Albania,AL
2021-03-12,138,4,109,114681,3026,79428,Algeria,DZ
2021-03-12,41,0,23,11130,112,10708,Andorra,AD
2021-03-12,47,0,84,21161,516,19761,Angola,AO
2021-03-12,20,1,34,882,23,489,Antigua and Barbuda,AG
2021-03-12,7693,107,5049,2169694,53359,1961640,Argentina,AR
2021-03-12,340,5,163,175538,3237,165259,Armenia,AM
2021-03-12,16,0,9,29090,909,22945,Australia,AU
2021-03-12,2528,19,2012,481919,8776,449053,Austria,AT


In [0]:
df_resumo_pais.printSchema()

root
-- DATE: date (nullable = true)
-- NEWCONFIRMED: long (nullable = true)
-- NEWDEATHS: long (nullable = true)
-- NEWRECOVERED: long (nullable = true)
-- TOTALCONFIRMED: long (nullable = true)
-- TOTALDEATHS: long (nullable = true)
-- TOTALRECOVERED: long (nullable = true)
-- NOME: string (nullable = true)
-- SIGLA: string (nullable = true)